# 0. Packages

In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.8 MB/s et

# 1. Cree la base de donnees

In [1]:
import chromadb

def init_db():
    """Initialisation de la base de données ChromaDB"""
    return chromadb.PersistentClient(path="/content/chromadb")

db = init_db()

# 2. Cree les Collections

In [2]:
roles = ["Simple", "Dev", "Admin"]
collections = {role: db.get_or_create_collection(name=f"qa_{role}") for role in roles}

In [3]:
collections

{'Simple': Collection(name=qa_Simple),
 'Dev': Collection(name=qa_Dev),
 'Admin': Collection(name=qa_Admin)}

In [4]:
db.list_collections()

['qa_Dev', 'qa_Simple', 'qa_Admin']

In [5]:
def Num_Data():
  for coll in collections:
      print('Num ', coll,' : ', collections[coll].count())

In [6]:
Num_Data()

Num  Simple  :  0
Num  Dev  :  0
Num  Admin  :  0


# 3. Add/ Ask questions

In [7]:
def add_document(role: str, question: str, answer: str):
    """Ajoute une question-réponse dans la collection associée au rôle spécifié"""
    if role not in collections:
        print("Rôle invalide!")
        return

    collection = collections[role]
    collection.add(
        documents=[question],
        metadatas=[{"answer": answer}],
        ids=[f"{role}_{question}"]  # Identifiant unique basé sur le rôle
    )
    print(f"Ajouté avec succès dans la collection {role}")

In [8]:
seuil = 0.1
message_question_existante_autre_role = "Cette question associée à un autre rôle."
message_hors_contexte = "Cette question ne correspond à aucun des rôles définis."

In [9]:
def ask_autre_role(role: str, question: str):
    for rol in collections:
        if rol != role:
            collection = collections[rol]
            results = collection.query(query_texts=[question], n_results=1)

            if results and results.get("documents"):
                distance = results["distances"][0][0] if results["distances"] else None
                if distance is not None and distance <= seuil:
                    return True
    return False  # Ajoute un return par défaut pour éviter des erreurs

In [10]:
def ask_question(role: str, question: str):
    """Recherche la réponse en fonction du rôle donné"""
    if role not in collections:
        return "Rôle invalide!"

    collection = collections[role]
    results = collection.query(query_texts=[question], n_results=1)

    if results and results.get("documents"):
        distance = results["distances"][0][0] if results["distances"] else None

        if distance <= seuil:
          answer = results["metadatas"][0][0]["answer"] if results["metadatas"] else "Réponse non disponible"
          #return f"Réponse: {answer}, Pertinence (distance): {distance:.4f}"
          return answer

        else :
          temp = ask_autre_role(role, question)
          if temp:
            return message_question_existante_autre_role
          else:
            return message_hors_contexte
          #return "Seuil elvee !"

        #return f"Réponse: {answer}, Pertinence (distance): {distance:.4f}"

    return "Aucune réponse trouvée"

In [19]:
# # Exemple d'utilisation
# add_document("Simple", "Quel est le capital du Maroc?", "Rabat")
# add_document("Dev", "Qu'est-ce que Python?", "Un langage de programmation interprété.")
# add_document("Admin", "Comment redémarrer un serveur Linux?", "Utiliser la commande 'sudo reboot'.")

In [20]:
# print(ask_question("Simple", "Quel est le capital du Maroc?"))  # Devrait retourner Rabat
# print(ask_question("Dev", "Qu'est-ce que Python?"))  # Devrait retourner la réponse associée
# print(ask_question("Admin", "Comment redémarrer un serveur Linux?"))  # Réponse Admin

In [21]:
# ask_question('Dev', "Qui est le Groupe OMF ?")

# 4. Creation du contexte

In [ ]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Initialize and constants

api_key = "##"

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI(api_key=api_key)

API key looks good so far


In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
ed = Website("https://www.groupe-omf.com/")
ed.links

['#',
 'https://www.groupe-omf.com/',
 'https://www.facebook.com/groupeomf',
 'https://www.twitter.com/groupeomf',
 'https://www.instagram.com/groupe_omf',
 'https://www.linkedin.com/company/groupe-omf',
 'https://www.groupe-omf.com/contact/',
 '#',
 'https://www.groupe-omf.com/',
 '#',
 '#',
 'https://www.groupe-omf.com/',
 'https://www.groupe-omf.com/groupe-omf/',
 'https://www.groupe-omf.com/mot-du-president/',
 'https://www.groupe-omf.com/missions-valeurs/',
 'https://www.groupe-omf.com/politique-qhse/',
 'https://www.groupe-omf.com/clients/',
 '/services',
 'https://www.groupe-omf.com/creation-de-societe/',
 'https://www.groupe-omf.com/tenue-de-comptabilite/',
 'https://www.groupe-omf.com/gestion-deleguee-de-la-paie/',
 'https://www.groupe-omf.com/assistance-iso/',
 'https://www.groupe-omf.com/expertise-rh/',
 'https://www.groupe-omf.com/optimisation-fiscale/',
 'https://www.groupe-omf.com/portage-salarial/',
 'https://www.groupe-omf.com/delocalisation/',
 'https://www.groupe-omf.

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [ ]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.groupe-omf.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#
https://www.groupe-omf.com/
https://www.facebook.com/groupeomf
https://www.twitter.com/groupeomf
https://www.instagram.com/groupe_omf
https://www.linkedin.com/company/groupe-omf
https://www.groupe-omf.com/contact/
#
https://www.groupe-omf.com/
#
#
https://www.groupe-omf.com/
https://www.groupe-omf.com/groupe-omf/
https://www.groupe-omf.com/mot-du-president/
https://www.groupe-omf.com/missions-valeurs/
https://www.groupe-omf.com/politique-qhse/
https://www.groupe-omf.com/clients/
/services
https://www.groupe-omf.com/creation-de-societe/
https://www.groupe-omf.com/tenue-de-comptabilite/
https://www.groupe-omf.com/gestion-deleguee-de-la-paie/
https://www.groupe-omf.com/assistance-iso/
https:

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://www.groupe-omf.com/")
huggingface.links

['#',
 'https://www.groupe-omf.com/',
 'https://www.facebook.com/groupeomf',
 'https://www.twitter.com/groupeomf',
 'https://www.instagram.com/groupe_omf',
 'https://www.linkedin.com/company/groupe-omf',
 'https://www.groupe-omf.com/contact/',
 '#',
 'https://www.groupe-omf.com/',
 '#',
 '#',
 'https://www.groupe-omf.com/',
 'https://www.groupe-omf.com/groupe-omf/',
 'https://www.groupe-omf.com/mot-du-president/',
 'https://www.groupe-omf.com/missions-valeurs/',
 'https://www.groupe-omf.com/politique-qhse/',
 'https://www.groupe-omf.com/clients/',
 '/services',
 'https://www.groupe-omf.com/creation-de-societe/',
 'https://www.groupe-omf.com/tenue-de-comptabilite/',
 'https://www.groupe-omf.com/gestion-deleguee-de-la-paie/',
 'https://www.groupe-omf.com/assistance-iso/',
 'https://www.groupe-omf.com/expertise-rh/',
 'https://www.groupe-omf.com/optimisation-fiscale/',
 'https://www.groupe-omf.com/portage-salarial/',
 'https://www.groupe-omf.com/delocalisation/',
 'https://www.groupe-omf.

In [ ]:
get_links("https://www.groupe-omf.com/")

{'links': [{'type': 'about page',
   'url': 'https://www.groupe-omf.com/groupe-omf/'},
  {'type': 'about page',
   'url': 'https://www.groupe-omf.com/mot-du-president/'},
  {'type': 'about page',
   'url': 'https://www.groupe-omf.com/missions-valeurs/'},
  {'type': 'careers page', 'url': 'https://www.groupe-omf.com/?page_id=1606'},
  {'type': 'contact page', 'url': 'https://www.groupe-omf.com/contact/'}]}

## Second step: make the brochure!


In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url) ### API
    #print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://www.groupe-omf.com/"))

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n8 days ago\n•\n155k\n•\n1.33k\ntencent/Hunyuan-A13B-Instruct\nUpdated\n4 days ago\n•\n13.7k\n•\n712\ngoogle/gemma-3n-E4B-it\nUpdated\n2 days ago\n•\n197k\n•\n458\nTHUDM/GLM-4.1V-9B-Thinking\nUpdated\n2 days ago\n•\n7.03k\n•\n210\nkyutai/tts-1.6b-en_fr\nUpdated\n2 days ago\n•\n7.19k\n•\n173\nBrowse 1M+ models\nSpaces\nRunning\n9.27k\n9.27k\nDee

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
Brochure = create_brochure("Groupe OMF", "https://www.groupe-omf.com/")

# 5. Generation du donnees en fonction du context de l'entreprise

In [ ]:
# Brochure de référence
brochure = """
Brochure de l'entreprise: Groupe OMF
Qui sommes-nous ?
Groupe OMF est une entreprise marocaine dynamique, engagée à accompagner les chefs d'entreprise dans leurs défis quotidiens. Forts de plus de 20 ans d'expérience, nous nous positionnons comme le partenaire de référence pour les entreprises cherchant des solutions personnalisées et de qualité dans divers domaines, allant de la gestion à la communication, en passant par la technologie.

Nos services
Nous proposons une large gamme de services, notamment :

Gestion: Aide à la planification, organisation et contrôle des ressources.
Optimisation: Amélioration de l'utilisation des ressources pour des résultats optimaux.
Formation: Développement des compétences des salariés pour renforcer la performance.
Conseil: Expertise pour des décisions stratégiques et organisationnelles.
Communication: Stratégies pour la diffusion efficace des messages d'entreprise.
Technologie: Outils et méthodes pour la gestion de l'information numérique.
Politique QHSE
Chez Groupe OMF, nous attachons une grande importance à la qualité, à l'hygiène, à la sécurité et à l'environnement. Nos engagements en matière de QHSE assurent un service de haute qualité et respectueux des normes.

Valeurs et culture d'entreprise
Nous croyons fermement en :

Respect: Chaque membre de notre équipe et chaque client méritent d'être respectés.
Exemplarité: Un comportement éthique et professionnel dans toutes nos interactions.
Innovation: Une quête constante pour s'adapter aux évolutions du marché.
Esprit d'équipe: Collaboration pour atteindre des objectifs communs.
Pour nos clients
Nous sommes dédiés à fournir des solutions adaptées aux besoins spécifiques de nos clients, en mettant l'accent sur la satisfaction client et l'innovation continue. Nos clients proviennent de divers secteurs et tirent parti de notre expertise pour atteindre leurs ambitions.

Carrières chez Groupe OMF
Nous recherchons des talents motivés et passionnés à rejoindre notre équipe. Notre processus de recrutement est rigoureux et vise à identifier des candidats qui partagent nos valeurs. Nous offrons des opportunités de développement professionnel grâce à des formations continues et un environnement de travail collaboratif.

Contactez-nous
Adresse:
Residence Abdelmoumen, Bd Raphael,
Casablanca 20102, Maroc

Téléphone:
+212 522 27 35 38

Email: Nous Contacter

Visitez notre site web pour plus d’informations sur nos services et opportunités de carrière.

Groupe OMF – Au service de l'entreprise, nous sommes là pour vous aider à prendre des mesures décisives et à obtenir des résultats durables.
"""

In [ ]:
system_prompt = "Tu es un expert en création de questionnaires d'entreprise. Tu dois retourner les résultats sous forme de liste Python de tuples [(question, réponse), ...]."

# Générateur de Questions/Réponses
def generate_qa(profile, type_question, brochure_text, nb=20):
    instruction = ""
    if type_question == "personnalisées":
        instruction = (
            f"Génère {nb} paires Question/Réponse pour un utilisateur de type '{profile}' concernant l'entreprise ci-dessous. "
            "Les réponses doivent être précises et compréhensibles selon le niveau de l'utilisateur."
        )
    elif type_question == "communes":
        instruction = (
            f"Génère {nb} questions communes sur l'entreprise ci-dessous. "
            "Pour chaque question, donne 3 réponses : une pour 'Simple', une pour 'Dev', et une pour 'Admin'."
        )
    elif type_question == "hors_contexte":
        instruction = (
            f"Génère {nb} paires Question/Réponse absurdes ou hors contexte, mais dans un format professionnel et amusant."
        )

    prompt = f"{instruction}\nVoici la brochure de l'entreprise :\n{brochure_text}"

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

## Questions / Reponses personnalisées

In [ ]:
# ⚙️ Exécution
questions_simple = generate_qa("Simple", "personnalisées", brochure)

In [ ]:
print(questions_simple)

```python
questions_reponses = [
    ("Qui est le Groupe OMF ?", "Le Groupe OMF est une entreprise marocaine dynamique qui accompagne les chefs d'entreprise depuis plus de 20 ans."),
    ("Quels services propose le Groupe OMF ?", "Le Groupe OMF propose des services en gestion, optimisation, formation, conseil, communication et technologie."),
    ("Quelle est l'expertise du Groupe OMF ?", "Le Groupe OMF offre une expertise pour aider à la prise de décisions stratégiques et organisationnelles."),
    ("Comment le Groupe OMF assure-t-il la qualité de ses services ?", "Le Groupe OMF respecte une politique QHSE qui met l'accent sur la qualité, l'hygiène, la sécurité et l'environnement."),
    ("Quelles sont les valeurs du Groupe OMF ?", "Les valeurs du Groupe OMF incluent le respect, l'exemplarité, l'innovation et l'esprit d'équipe."),
    ("Où est situé le Groupe OMF ?", "Le Groupe OMF est situé à Residence Abdelmoumen, Bd Raphael, Casablanca 20102, Maroc."),
    ("Quel est le numéro de t

In [ ]:
# 20 Questions spécifiques par Simple
questions_dev = generate_qa("Dev", "personnalisées", brochure)

In [ ]:
questions_dev = """
Voici une liste de 20 paires de questions et réponses adaptées à un utilisateur de type 'Dev' concernant le Groupe OMF :

```python
questions_reponses  = [
    ("Quelle est la mission principale du Groupe OMF ?", "Accompagner les chefs d'entreprise dans leurs défis quotidiens avec des solutions personnalisées."),
    ("Depuis combien d'années le Groupe OMF est-il actif ?", "Plus de 20 ans d'expérience."),
    ("Quels types de services propose le Groupe OMF ?", "Gestion, optimisation, formation, conseil, communication, et technologie."),
    ("Comment le Groupe OMF aide-t-il à la gestion des ressources ?", "Il fournit de l'aide à la planification, l'organisation et le contrôle des ressources."),
    ("Qu'entend-on par optimisation au sein du Groupe OMF ?", "Amélioration de l'utilisation des ressources pour des résultats optimaux."),
    ("Le Groupe OMF propose-t-il des formations ?", "Oui, pour développer les compétences des salariés et renforcer la performance."),
    ("Quel type de conseil est offert par le Groupe OMF ?", "Conseil stratégique et organisationnel basé sur l'expertise."),
    ("Comment le Groupe OMF aborde-t-il la communication ?", "En proposant des stratégies pour la diffusion efficace des messages d'entreprise."),
    ("Quels outils technologiques fournit le Groupe OMF ?", "Des outils et méthodes pour la gestion de l'information numérique."),
    ("Qu'est-ce que la politique QHSE au sein du Groupe OMF ?", "Un engagement envers la qualité, l'hygiène, la sécurité et l'environnement."),
    ("Quelles valeurs sont prônées par le Groupe OMF ?", "Respect, exemplarité, innovation, esprit d'équipe."),
    ("Comment le Groupe OMF garantit-il la satisfaction client ?", "En fournissant des solutions adaptées aux besoins spécifiques des clients."),
    ("Dans quels secteurs les clients du Groupe OMF opèrent-ils ?", "Divers secteurs, profitant de notre expertise pour atteindre leurs ambitions."),
    ("Quel type de candidats le Groupe OMF recherche-t-il pour ses carrières ?", "Des talents motivés et passionnés partageant les valeurs de l'entreprise."),
    ("Comment se déroule le processus de recrutement chez Groupe OMF ?", "Rigorieux, visant à identifier des candidats alignés avec nos valeurs."),
    ("Le Groupe OMF offre-t-il des opportunités de développement professionnel ?", "Oui, grâce à des formations continues et un environnement collaboratif."),
    ("Quelle est l'adresse du Groupe OMF ?", "Residence Abdelmoumen, Bd Raphael, Casablanca 20102, Maroc."),
    ("Quel est le numéro de téléphone du Groupe OMF ?", "+212 522 27 35 38."),
    ("Comment peut-on contacter le Groupe OMF par email ?", "En utilisant le lien 'Nous Contacter' sur leur site."),
    ("Quel est l'objectif global du Groupe OMF ?", "Aider les entreprises à prendre des mesures décisives et à obtenir des résultats durables."),
]
```
"""

In [ ]:
print(questions_dev)


Voici une liste de 20 paires de questions et réponses adaptées à un utilisateur de type 'Dev' concernant le Groupe OMF :

```python
questions_reponses  = [
    ("Quelle est la mission principale du Groupe OMF ?", "Accompagner les chefs d'entreprise dans leurs défis quotidiens avec des solutions personnalisées."),
    ("Depuis combien d'années le Groupe OMF est-il actif ?", "Plus de 20 ans d'expérience."),
    ("Quels types de services propose le Groupe OMF ?", "Gestion, optimisation, formation, conseil, communication, et technologie."),
    ("Comment le Groupe OMF aide-t-il à la gestion des ressources ?", "Il fournit de l'aide à la planification, l'organisation et le contrôle des ressources."),
    ("Qu'entend-on par optimisation au sein du Groupe OMF ?", "Amélioration de l'utilisation des ressources pour des résultats optimaux."),
    ("Le Groupe OMF propose-t-il des formations ?", "Oui, pour développer les compétences des salariés et renforcer la performance."),
    ("Quel type de c

In [ ]:
questions_admin = generate_qa("Admin", "personnalisées", brochure)

In [ ]:
print(questions_admin)

## Questions / Reponses Communes

In [ ]:
# 20 Questions communes avec réponses multiples
questions_communes = generate_qa("Tous", "communes", brochure)

In [ ]:
print(questions_communes)

## Questions / Reponses Absurdes

In [ ]:
questions_absurdes = generate_qa("Tous", "hors_contexte", brochure)

In [ ]:
print(questions_absurdes)

In [ ]:
questions_absurdes = """
Voici une liste de 20 paires de questions/réponses absurdes ou hors contexte que vous pourriez utiliser de manière professionnelle et amusante :

```python
absurd_questions_answers = [
    ("Si une licorne pouvait conseiller sur la gestion des ressources, que dirait-elle ?", "Toujours garder une réserve de paille, c'est essentiel pour la motivation."),
    ("Quelle stratégie de communication utiliser pour vendre des glaces dans le désert ?", "Un mélange de sable et de crème chantilly pourrait faire des merveilles."),
    ("Comment optimiser les performances d'un hamster en roue ?", "Installer des panneaux solaires et un café à volonté."),
    ("Que signifie QHSE pour les robots dansant ?", "Qualité, Hyperbole, Sécurité Électrisante."),
    ("Pourquoi les chats sont-ils de très mauvais conseillers en carrière ?", "Ils préfèrent dormir sur vos décisions plutôt que de les soutenir."),
    ("Si l'innovation était un fruit, lequel serait-il ?", "Une mangue qui chante des chansons d'été."),
    ("Quelle est la meilleure façon de gérer une réunion avec des extraterrestres ?", "Prévoir un buffet intergalactique et des diapositives sur les tendances terriennes."),
    ("Comment évaluer la performance d'une équipe de pingouins ?", "Compter le nombre de glissades réussies sur la banquise."),
    ("Quel est l'impact des couleurs des cravates sur le moral d'un cactus ?", "Une cravate rouge booste la photosynthèse, paraît-il."),
    ("Comment réduire le stress d'un cactus au travail ?", "Offrir des séances de méditation sous un soleil radieux."),
    ("Quel est le secret de la réussite d'un poisson d'entreprise ?", "Un bon réseau aquatique et un sens inné des courants."),
    ("Si les ordinateurs pouvaient avoir des émotions, que diraient-ils en réunion ?", "J'ai besoin d'un reboot émotionnel avant de continuer."),
    ("Pourquoi la pâtisserie est-elle essentielle pour la gestion du temps ?", "Les macarons bipolaires sont une excellente façon de rythmer la journée."),
    ("Si nos valeurs d'entreprise étaient des super-héros, qui seraient-ils ?", "Le Respect aurait un costume vert, l’Innovation porterait un masque futuriste."),
    ("Quel serait le rôle d'un perroquet dans notre équipe de formation ?", "Remplacer le PowerPoint par des imitations de vos feedbacks."),
    ("Que feriez-vous si un message de l'espace arrivait par erreur ?", "Le traiter comme une opportunité de communication innovante."),
    ("Quel est le plus grand défi dans le domaine de la technologie ?", "Convaincre un toaster de connaître ses limites."),
    ("Comment gérer un projet avec des gnomes de jardin ?", "Règle n°1 : ne jamais leur tourner le dos pendant la phase de planification."),
    ("Quel est l'intérêt d'une réunion avec des nuages ?", "Développer des stratégies pour une couverture pluvieuse de qualité."),
    ("Si les perroquets pouvaient postuler, quel serait leur atout le plus convaincant ?", "Une parole en or pour chaque proposition.")
]
```

Ces paires de questions et réponses ajoutent une touche humoristique tout en conservant un format pouvant être utilisé dans un contexte professionnel.

"""

# 6. Creation de la DataSet

## Nettoyer les listes

In [ ]:
import re
import ast

def extract_python_list(response_text):
    """
    Extrait et convertit une liste Python présente dans un texte brut.
    :param response_text: Chaîne contenant la liste au format texte
    :return: La liste Python extraite, ou None si extraction échoue
    """
    # Chercher le bloc ```python ... ```
    pattern = r"```python\s*(.*?)```"
    match = re.search(pattern, response_text, re.DOTALL)

    if not match:
        print("⚠️ Bloc Python non trouvé.")
        return None

    code_block = match.group(1)

    # Chercher la partie contenant la liste
    list_pattern = r"=\s*(\[[\s\S]*\])"
    list_match = re.search(list_pattern, code_block)

    if not list_match:
        print("⚠️ Liste non trouvée dans le bloc de code.")
        return None

    list_str = list_match.group(1)

    try:
        extracted_list = ast.literal_eval(list_str)
        return extracted_list
    except Exception as e:
        print(f"Erreur lors du parsing de la liste : {e}")
        return None

In [ ]:
questions_simple_Clean = extract_python_list(questions_simple)

In [ ]:
len(questions_simple_Clean)

In [ ]:
questions_dev_Clean = extract_python_list(questions_dev)

In [ ]:
len(questions_dev_Clean)

In [ ]:
questions_admin_Clean = extract_python_list(questions_admin)

In [ ]:
len(questions_admin_Clean)

In [ ]:
questions_communes_Clean = extract_python_list(questions_communes)

In [ ]:
len(questions_communes_Clean)

In [ ]:
questions_absurdes_Clean = extract_python_list(questions_absurdes)

In [ ]:
len(questions_absurdes_Clean)

## Cree l'objet Pandas

In [ ]:
import pandas as pd

# Initialisation liste finale
data = []

In [ ]:
# Simple
for q, r in questions_simple_Clean:
    data.append({"Question": q, "Réponse": r, "Profil": "Simple"})

# Dev
for q, r in questions_dev_Clean:
    data.append({"Question": q, "Réponse": r, "Profil": "Dev"})

# Admin
for q, r in questions_admin_Clean:
    data.append({"Question": q, "Réponse": r, "Profil": "Admin"})

# Communes : une ligne par réponse et profil
for q, r_simple, r_dev, r_admin in questions_communes_Clean:
    data.append({"Question": q, "Réponse": r_simple, "Profil": "Commun_Simple"})
    data.append({"Question": q, "Réponse": r_dev, "Profil": "Commun_Dev"})
    data.append({"Question": q, "Réponse": r_admin, "Profil": "Commun_Admin"})

# Absurdes
for q, r in questions_absurdes_Clean:
    data.append({"Question": q, "Réponse": r, "Profil": "Absurde"})

In [ ]:
# Création du DataFrame final
df_total = pd.DataFrame(data)

In [ ]:
df_total.shape

In [ ]:
# Aperçu
df_total.head()

In [ ]:
df_total['Profil'].unique()

In [ ]:
# Optionnel : Sauvegarde
df_total.to_csv("Questions_Reponses_Global_Contextialiser.csv", index=False)

# 7. Insertion des donnees dans ChromaDb

In [11]:
import pandas as pd

dataset = pd.read_csv("Questions_Reponses_Global_Contextialiser.csv")

In [12]:
dataset.shape

(141, 3)

In [13]:
dataset.columns

Index(['Question', 'Réponse', 'Profil'], dtype='object')

In [14]:
dataset['Profil'].unique()

array(['Simple', 'Dev', 'Admin', 'Commun_Simple', 'Commun_Dev',
       'Commun_Admin', 'Absurde'], dtype=object)

In [15]:
dataset['Question'].unique().shape

(95,)

In [16]:
for profil in dataset['Profil'].unique():
    print(f"\n****** Questions/Réponses uniques pour le profil : {profil} ******")

    # On récupère les paires uniques (question, réponse) par profil
    df_profil = dataset[dataset['Profil'] == profil][['Question', 'Réponse']].drop_duplicates()

    for _, row in df_profil.iterrows():
        print(f"Question : {row['Question']}\nRéponse : {row['Réponse']}\n")


****** Questions/Réponses uniques pour le profil : Simple ******
Question : Qui est le Groupe OMF ?
Réponse : Le Groupe OMF est une entreprise marocaine qui aide les chefs d'entreprise à relever leurs défis quotidiens.

Question : Depuis combien d'années le Groupe OMF est-il actif ?
Réponse : Nous avons plus de 20 ans d'expérience dans notre domaine.

Question : Quels types de services propose le Groupe OMF ?
Réponse : Nous proposons des services en gestion, optimisation, formation, conseil, communication et technologie.

Question : Que signifie le service de gestion chez Groupe OMF ?
Réponse : Le service de gestion aide à la planification, organisation et contrôle des ressources d'une entreprise.

Question : Quel est l'objectif de l'optimisation proposée par le Groupe OMF ?
Réponse : L'optimisation vise à améliorer l'utilisation des ressources pour des résultats optimaux.

Question : Pourquoi la formation est-elle importante pour le Groupe OMF ?
Réponse : La formation développe les c

In [28]:
dataset.loc[dataset['Question'] == "Quels types de services propose le Groupe OMF ?"]

,Question,Réponse,Profil
2,Quels types de services propose le Groupe OMF ?,"Nous proposons des services en gestion, optimi...",Simple
22,Quels types de services propose le Groupe OMF ?,"Gestion, optimisation, formation, conseil, com...",Dev
42,Quels types de services propose le Groupe OMF ?,"Gestion, optimisation, formation, conseil, com...",Admin


## 7.1 Simple utilisateur

In [17]:
#add_document("Simple", "Quelle est la capitale de la France ?", "Paris")

In [18]:
for question, reponse in dataset.loc[dataset['Profil'] == 'Simple', ['Question', 'Réponse']].values:
    add_document("Simple", question, reponse)

Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple


In [19]:
collections['Simple'].count()

20

## 7.2 Dev utilisateur

In [20]:
# add_document("Dev", "Qu'est-ce qu'une API ?", "Interface de programmation permettant aux applications de communiquer entre elles.")

In [21]:
for question, reponse in dataset.loc[dataset['Profil'] == 'Dev', ['Question', 'Réponse']].values:
    add_document("Dev", question, reponse)

Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev


In [22]:
collections['Dev'].count()

20

## 7.3 Admin utilisateur

In [23]:
# add_document("Admin", "Comment installer Apache sur un serveur Linux ?", "Utiliser la commande 'sudo apt install apache2' sur une machine Ubuntu.")

In [24]:
for question, reponse in dataset.loc[dataset['Profil'] == 'Admin', ['Question', 'Réponse']].values:
    add_document("Admin", question, reponse)

Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin


In [25]:
collections['Admin'].count()

21

## 7.4 Question communes

In [26]:
# # Question 1
# add_document("Simple", "Qu'est-ce que le cloud ?", "Un espace de stockage en ligne pour conserver des fichiers accessibles partout.")
# add_document("Dev", "Qu'est-ce que le cloud ?", "Une infrastructure qui permet d'héberger et d'exécuter des applications à distance.")
# add_document("Admin", "Qu'est-ce que le cloud ?", "Un ensemble de serveurs distants offrant des services de stockage, de calcul et de réseau.")

In [27]:
for question, reponse in dataset.loc[dataset['Profil'] == 'Commun_Simple', ['Question', 'Réponse']].values:
    add_document("Simple", question, reponse)

Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple
Ajouté avec succès dans la collection Simple


In [28]:
collections['Simple'].count()

40

In [29]:
for question, reponse in dataset.loc[dataset['Profil'] == 'Commun_Dev', ['Question', 'Réponse']].values:
    add_document("Dev", question, reponse)

Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev
Ajouté avec succès dans la collection Dev


In [30]:
collections['Dev'].count()

40

In [31]:
for question, reponse in dataset.loc[dataset['Profil'] == 'Commun_Admin', ['Question', 'Réponse']].values:
    add_document("Admin", question, reponse)

Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin
Ajouté avec succès dans la collection Admin


In [32]:
collections['Admin'].count()

41

# 6. Test Retriver Database

In [33]:
dataset.columns

Index(['Question', 'Réponse', 'Profil'], dtype='object')

In [34]:
dataset['Profil'].unique()

array(['Simple', 'Dev', 'Admin', 'Commun_Simple', 'Commun_Dev',
       'Commun_Admin', 'Absurde'], dtype=object)

In [35]:
dataset['Profil'] = dataset['Profil'].replace({
    'Commun_Simple': 'Simple',
    'Commun_Dev': 'Dev',
    'Commun_Admin': 'Admin'
})

In [36]:
dataset['Profil'].unique()

array(['Simple', 'Dev', 'Admin', 'Absurde'], dtype=object)

In [37]:
dataset[10:]

,Question,Réponse,Profil
10,Quelles sont les valeurs clés du Groupe OMF ?,"Les valeurs clés incluent le respect, l'exempl...",Simple
11,Comment le Groupe OMF assure-t-il la satisfact...,Nous fournissons des solutions adaptées aux be...,Simple
12,Quels secteurs d'activité bénéficient des serv...,"Nos clients proviennent de divers secteurs, to...",Simple
13,Comment puis-je postuler pour une carrière che...,Nous recherchons des talents motivés et passio...,Simple
14,Quelles opportunités de formation le Groupe OM...,Nous offrons des opportunités de développement...,Simple
...,...,...,...
136,Que feriez-vous si un message de l'espace arri...,Le traiter comme une opportunité de communicat...,Absurde
137,Quel est le plus grand défi dans le domaine de...,Convaincre un toaster de connaître ses limites.,Absurde
138,Comment gérer un projet avec des gnomes de jar...,Règle n°1 : ne jamais leur tourner le dos pend...,Absurde
139,Quel est l'intérêt d'une réunion avec des nuag...,Développer des stratégies pour une couverture ...,Absurde


In [74]:
# Identifier les questions dupliquées (peu importe le rôle)
dupliques = dataset[dataset.duplicated(subset=['Question'], keep=False)]

# Tri facultatif pour regrouper par question
dupliques = dupliques.sort_values(by='Question')

# Affichage des colonnes souhaitées
dupliques[['Question', 'Réponse', 'Profil']]

,Question,Réponse,Profil
70,Comment Groupe OMF s'engage-t-elle en matière ...,En suivant des normes strictes,Simple
72,Comment Groupe OMF s'engage-t-elle en matière ...,En respectant des engagements QHSE qui garanti...,Admin
71,Comment Groupe OMF s'engage-t-elle en matière ...,En adoptant des normes QHSE rigoureuses,Dev
111,Comment décrit-on l'importance des valeurs che...,Essentielles pour maintenir un bon environneme...,Admin
110,Comment décrit-on l'importance des valeurs che...,Essentielles pour la culture d'entreprise,Dev
...,...,...,...
65,Quels types de services propose Groupe OMF ?,"Gestion, optimisation, formation, technologie,...",Dev
66,Quels types de services propose Groupe OMF ?,"Gestion, optimisation, formation, conseil, tec...",Admin
2,Quels types de services propose le Groupe OMF ?,"Nous proposons des services en gestion, optimi...",Simple
22,Quels types de services propose le Groupe OMF ?,"Gestion, optimisation, formation, conseil, com...",Dev


In [62]:
# dataset.loc[dataset['Profil'].isin(['Commun_Simple', 'Commun_Dev', 'Commun_Admin']), ['Question', 'Réponse']]

In [72]:
seuil = 0

In [39]:
# ask_question('Dev', "Comment Groupe OMF s'engage-t-elle en matière de QHSE ?")

In [65]:
import numpy as np

# Exemple de seuils à tester (ajuste les valeurs selon ton besoin)
liste_seuils = np.arange(0, 1.05, 0.05)  # de 0.05 à 1.0 par pas de 0.05

In [66]:
liste_seuils

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

In [71]:
meilleur_seuil = None
meilleure_precision = 0

resultats_seuils = []

# Boucle sur les seuils
for seuil_test in liste_seuils:
    seuil = seuil_test  # Met à jour le seuil global dans tes fonctions
    total = 0
    correct = 0

    # Parcours de la dataset pour tester
    for _, row in dataset.iterrows():
        role = row['Profil']
        if role == 'Absurde':
          continue
        question = row['Question']
        reponse_attendue = row['Réponse']

        reponse_model = ask_question(role, question)

        print('reponse_model = ', reponse_model, ' ; reponse_attendue = ', reponse_attendue, ' \n')
        if reponse_model == reponse_attendue:
          print('reponse_model == reponse_attendue \n\n')
          correct += 1
        total += 1

    precision = correct / total if total > 0 else 0
    resultats_seuils.append( (seuil, precision) )

    print(f"******** Seuil : {seuil:.2f} - Précision : {precision:.4f}")

    # Mise à jour du meilleur seuil
    if precision > meilleure_precision:
        meilleure_precision = precision
        meilleur_seuil = seuil

reponse_model =  Le Groupe OMF est une entreprise marocaine qui aide les chefs d'entreprise à relever leurs défis quotidiens.  ; reponse_attendue =  Le Groupe OMF est une entreprise marocaine qui aide les chefs d'entreprise à relever leurs défis quotidiens.  

reponse_model == reponse_attendue 


reponse_model =  Nous avons plus de 20 ans d'expérience dans notre domaine.  ; reponse_attendue =  Nous avons plus de 20 ans d'expérience dans notre domaine.  

reponse_model == reponse_attendue 


reponse_model =  Nous proposons des services en gestion, optimisation, formation, conseil, communication et technologie.  ; reponse_attendue =  Nous proposons des services en gestion, optimisation, formation, conseil, communication et technologie.  

reponse_model == reponse_attendue 


reponse_model =  Le service de gestion aide à la planification, organisation et contrôle des ressources d'une entreprise.  ; reponse_attendue =  Le service de gestion aide à la planification, organisation et contrôle

KeyboardInterrupt: 

In [ ]:
print("\n==== Meilleur Résultat ====")
print(f"Meilleur seuil : {meilleur_seuil:.2f} - Précision : {meilleure_precision:.4f}")

# 6. Data Set de Test

In [ ]:
import json

# Fonction utilitaire pour créer les entrées du dataset
def create_dataset(pairs, role, is_ooc=False):
    dataset = []
    for question, answer in pairs:
        dataset.append({
            "question": question,
            "expected_answer": (
                "Cette question ne correspond à aucun des rôles définis."
                if is_ooc else answer
            ),
            "role": role
        })
    return dataset

In [ ]:
# 1. Données simples
list_simple = [
    ("Quelle est la capitale de la France ?", "Paris"),
    ("Quel est le plus grand océan du monde ?", "Océan Pacifique"),
    ("Combien de continents y a-t-il sur Terre ?", "7"),
    ("Qui a écrit 'Les Misérables' ?", "Victor Hugo"),
    ("Quelle est la monnaie officielle des États-Unis ?", "Dollar américain"),
    ("Quel est l'animal national de l'Australie ?", "Kangourou"),
    ("Quelle est la langue officielle du Brésil ?", "Portugais"),
    ("Quelle est la hauteur du Mont Everest ?", "8 848 mètres"),
    ("Dans quel pays se trouve le Taj Mahal ?", "Inde"),
    ("Quelle est la couleur du drapeau du Japon ?", "Rouge et blanc")
]
list_simple_re = [
    ("Quelle est la ville qui est la capitale de la France ?", "Paris"),
    ("Quel est l'océan ayant la plus grande superficie ?", "Océan Pacifique"),
    ("Combien de continents sont présents sur notre planète ?", "7"),
    ("Quel auteur a écrit le livre 'Les Misérables' ?", "Victor Hugo"),
    ("Quelle devise est utilisée aux États-Unis ?", "Dollar américain"),
    ("Quel est l'animal emblématique de l'Australie ?", "Kangourou"),
    ("Quel est le principal langage parlé au Brésil ?", "Portugais"),
    ("Quelle est l'altitude du Mont Everest ?", "8 848 mètres"),
    ("Où peut-on trouver le Taj Mahal ?", "Inde"),
    ("Quelles couleurs composent le drapeau japonais ?", "Rouge et blanc")
]

# 2. Données dev
list_dev = [
    ("Qu'est-ce qu'une API ?", "Interface de programmation permettant aux applications de communiquer entre elles."),
    ("Quelle est la différence entre un tableau et une liste en Python ?", "Un tableau a une taille fixe, alors qu'une liste est dynamique."),
    ("Qu'est-ce qu'un framework ?", "Un ensemble d'outils et de bibliothèques permettant de développer des applications rapidement."),
    ("Quelle est la fonction de Git ?", "Git est un système de contrôle de version distribué."),
    ("Qu'est-ce qu'une base de données relationnelle ?", "Une base de données où les données sont organisées en tables."),
    ("À quoi sert un fichier .env ?", "Il est utilisé pour stocker des variables d'environnement sensibles."),
    ("Qu'est-ce que le versioning de code ?", "Le versioning permet de gérer les différentes versions d'un code source."),
    ("Quelle est la différence entre une méthode GET et POST en HTTP ?", "GET est utilisé pour récupérer des données, POST pour envoyer des données."),
    ("Que fait une instruction if en programmation ?", "Elle permet d'exécuter du code conditionnel."),
    ("Qu'est-ce que le principe DRY en développement ?", "DRY signifie 'Don't Repeat Yourself', éviter la duplication du code.")
]

list_dev_re = [
    ("Qu'est-ce qu'une API et à quoi sert-elle ?", "Une API permet aux applications de communiquer entre elles."),
    ("Quelle distinction existe entre un tableau et une liste en Python ?", "Un tableau a une taille fixe, tandis qu'une liste peut être redimensionnée."),
    ("En quoi consiste un framework en développement ?", "Un framework est un ensemble d'outils pour faciliter le développement d'applications."),
    ("Comment Git facilite-t-il la gestion de versions ?", "Git permet de suivre les changements et de gérer les versions du code source."),
    ("Qu'est-ce qu'une base de données relationnelle et comment fonctionne-t-elle ?", "Une base de données relationnelle stocke des données sous forme de tables reliées entre elles."),
    ("À quoi sert un fichier .env dans un projet ?", "Un fichier .env contient des variables d'environnement, souvent utilisées pour stocker des secrets."),
    ("Qu'entend-on par versioning dans le développement de logiciels ?", "Le versioning permet de gérer différentes versions d'un projet de manière organisée."),
    ("Quelle est la différence entre les méthodes HTTP GET et POST ?", "GET est utilisé pour récupérer des données, tandis que POST est utilisé pour envoyer des données."),
    ("Que permet d'accomplir une instruction 'if' dans un programme ?", "L'instruction 'if' permet d'exécuter un bloc de code conditionnellement."),
    ("Quel principe est exprimé par l'acronyme DRY en développement logiciel ?", "DRY signifie 'Don't Repeat Yourself', il vise à éviter la duplication du code.")
]

# 3. Données admin
list_admin = [
    ("Comment installer Apache sur un serveur Linux ?", "Utiliser la commande 'sudo apt install apache2' sur une machine Ubuntu."),
    ("Que faire en cas de 'Disk Full' sur un serveur Linux ?", "Libérer de l'espace disque en supprimant des fichiers inutiles."),
    ("Comment vérifier les journaux système sur un serveur Linux ?", "Utiliser la commande 'journalctl' pour consulter les journaux."),
    ("Comment configurer un pare-feu sur Ubuntu ?", "Utiliser 'ufw' pour configurer un pare-feu."),
    ("Quelle commande permet de redémarrer un service sous Linux ?", "Utiliser la commande 'sudo systemctl restart <service>'."),
    ("Comment ajouter un utilisateur sur un serveur Linux ?", "Utiliser la commande 'sudo adduser <nom_utilisateur>'."),
    ("Comment configurer un serveur DNS sur Linux ?", "Modifier le fichier '/etc/resolv.conf' pour spécifier les serveurs DNS."),
    ("Que fait la commande chmod ?", "Elle permet de modifier les permissions d'un fichier ou d'un répertoire."),
    ("Comment configurer un serveur Nginx pour une application web ?", "Modifier les fichiers de configuration dans '/etc/nginx/sites-available' et activer le site."),
    ("Que faire pour sécuriser un serveur Linux contre les attaques DDoS ?", "Utiliser des outils comme fail2ban et configurer un pare-feu adapté.")
]

list_admin_re = [
    ("Comment procéder pour installer Apache sur un serveur Linux ?", "Utilisez la commande 'sudo apt install apache2' sur une machine Ubuntu."),
    ("Que faire lorsqu'un serveur Linux affiche un message indiquant que l'espace disque est plein ?", "Libérez de l'espace disque en supprimant des fichiers inutiles ou en agrandissant la partition."),
    ("Comment accéder et analyser les journaux systèmes sur un serveur Linux ?", "Utilisez la commande 'journalctl' pour consulter les journaux système."),
    ("Quelle est la méthode pour configurer un pare-feu sur un serveur Ubuntu ?", "Utilisez 'ufw' pour activer et configurer le pare-feu sur Ubuntu."),
    ("Quelle commande faut-il utiliser pour redémarrer un service sur Linux ?", "La commande est 'sudo systemctl restart <service>'."),
    ("Quelle procédure faut-il suivre pour ajouter un utilisateur sur un serveur Linux ?", "Utilisez la commande 'sudo adduser <nom_utilisateur>' pour ajouter un utilisateur."),
    ("Comment configurer un serveur DNS sous Linux ?", "Modifiez le fichier '/etc/resolv.conf' pour spécifier les serveurs DNS."),
    ("Que permet de faire la commande chmod sur Linux ?", "La commande 'chmod' permet de modifier les permissions d'un fichier ou répertoire."),
    ("Comment mettre en place un serveur Nginx pour héberger une application web ?", "Modifiez les fichiers dans '/etc/nginx/sites-available' et activez le site avec un lien symbolique."),
    ("Quelles mesures prendre pour protéger un serveur Linux contre les attaques par déni de service (DDoS) ?", "Utilisez des outils comme fail2ban et configurez un pare-feu pour limiter les connexions malveillantes.")
]

# 4. Données hors-contexte
list_questions_hors = [
    ("Quelle est la durée d'un jour sur Mars ?", "24 heures 39 minutes"),
    ("Qui a inventé l'ampoule électrique ?", "Thomas Edison"),
    ("Qu'est-ce que le Big Bang ?", "La théorie qui décrit l'origine de l'univers à partir d'un point très dense et chaud."),
    ("Quelle est la plus grande forêt du monde ?", "La forêt amazonienne"),
    ("Qu'est-ce qu'une éclipse solaire ?", "Un phénomène où la Lune passe entre la Terre et le Soleil."),
    ("Quel est l'animal le plus rapide du monde ?", "Le guépard"),
    ("Qu'est-ce que la gravité ?", "La force qui attire les objets vers le centre de la Terre."),
    ("En quelle année l'Homme a-t-il marché sur la Lune pour la première fois ?", "1969"),
    ("Quel est le pays le plus peuplé du monde ?", "La Chine"),
    ("Qu'est-ce que l'effet de serre ?", "Le phénomène par lequel certains gaz emprisonnent la chaleur dans l'atmosphère terrestre.")
]

In [ ]:
# Création du dataset complet
dataset = []
dataset += create_dataset(list_simple, "Simple")
dataset += create_dataset(list_simple_re, "Simple")
dataset += create_dataset(list_dev, "Dev")
dataset += create_dataset(list_dev_re, "Dev")
dataset += create_dataset(list_admin, "Admin")
dataset += create_dataset(list_admin_re, "Admin")
dataset += create_dataset(list_questions_hors, "Simple", is_ooc=True)
dataset += create_dataset(list_questions_hors, "Dev", is_ooc=True)
dataset += create_dataset(list_questions_hors, "Admin", is_ooc=True)

In [ ]:
# Enregistrer dans un fichier JSON
with open("eval_dataset.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=2)

print("✅ Dataset généré avec succès : rag_eval_dataset.json")

#7. Models : LLMs

In [45]:
from huggingface_hub import login
login(new_session=False)

In [46]:
# !pip install bitsandbytes transformers accelerate
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

##7.1 HuggingFaceH4/zephyr-7b-beta

In [ ]:
# from huggingface_hub import login
# login()

il est préférable de le quantifier pour optimiser les performances et réduire la consommation mémoire, surtout si tu tournes sur un GPU avec peu de VRAM.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Configuration de quantization 4-bit
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Charger Zephyr 7B Beta
model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16
)

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import torch

# # Modèle de traduction multilingue
# model_name = "facebook/mbart-large-50-many-to-many-mmt"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

In [ ]:
# tokenizer_Llama3.pad_token_id = tokenizer_Llama3.eos_token_id

In [ ]:
# Vérifier si le modèle est bien chargé sur GPU
import torch

# Vérifie si CUDA est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Le modèle sera chargé sur : {device}")

# Envoie explicitement le modèle sur le GPU
model = model.to(device)

# Vérifie où se trouve une couche du modèle
print(next(model.parameters()).device)

In [ ]:
prompt = "Translate this sentence from English to French: I love machine learning"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Génération
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("✅ Traduction :", result)

In [ ]:
import torch

# Charger le modèle sur le GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

def query_Model_zephyr(question, reponse):
    # Format the input prompt
    #formatted_prompt = f"Question: {question}\n\nRespons: {response}"
    formatted_prompt = f"""
                              Voici une question posée par un utilisateur et la réponse récupérée à partir de ChromaDB.

                              🔹 **Instructions claires :**
                              - **Si la réponse récupérée est valide**, reformule-la en français avec plus de contexte et d'explications. Ajoute des émojis dynamiquement pour illustrer les concepts (ex : 🌍 pour un lieu, 🧠 pour une explication, ✅ pour une réponse correcte). Termine la réponse par ✅.
                              - **Si la réponse est exactement "Cette question ne correspond à aucun des rôles définis.", alors affiche exactement :**
                                **"Cette question ne correspond à aucun des rôles définis. ❌"**
                              - **Si la réponse est exactement " Cette question associée à un autre rôle. ", alors affiche exactement :**
                                **"Cette question est associée à un autre rôle. 🔄"**
                              - **Ne génère aucun autre texte en dehors des trois cas définis ci-dessus.**

                              **⚠️ Attention :** Tu dois suivre ces règles strictement et ne pas ajouter d'explications supplémentaires si la réponse ne correspond pas aux données disponibles.

                              **Longueur maximale de la réponse générée : 500 caractères.**

                              Question : {question}
                              Réponse récupérée : {reponse}
                              💡 Réponse en français :
                        """


    # Tokenization
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

    # Génération de la réponse
    outputs = model.generate(**inputs, max_new_tokens=100)

    # Décodage du résultat
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

In [ ]:
llm_response = query_Model_zephyr("Quelle est la capitale de la France ?", "Paris")
llm_response

In [ ]:
def query_zephyr(query:str, role:str):
  #print('query : ', query)
  #print('role : ', role)
  response = ask_question(role, query)
  print('\nresponse ask_question : ', response)

  if response == message_question_existante_autre_role:
    return response + ' 🔄'
  elif response==message_hors_contexte:
    return response + ' ❌'
  else:
    test_answer = query_Model_zephyr(query, response)

    # Extraction de la réponse en français après "💡 Réponse en français :"
    start_index = test_answer.find("💡 Réponse en français :") + len("💡 Réponse en français :")
    filtered_response = test_answer[start_index:].strip()

    # Enlever "Correcte !" à la fin si nécessaire
    filtered_response = filtered_response.replace("Correct", "").strip()
    print('\nfiltered_response : ', filtered_response)

    return filtered_response

### 6.1.1 Simple Utilisateur

In [ ]:
query = "Quelle est la capitale de la France ?"
role = "Simple"

In [ ]:
Retrivel_response = ask_question(role, query)
Retrivel_response

In [ ]:
llm_response = query_Model_zephyr(query, Retrivel_response)
llm_response

In [ ]:
query_zephyr("Quelle est la capitale de la France ?", "Simple")

### 6.1.2 Dev Utilisateur

In [ ]:
query_Mistral7("Qu'est-ce qu'une API ?", "Dev")

### 6.1.3 Admin Utilisateur

In [ ]:
query_Mistral7("Comment installer Apache sur un serveur Linux ?", "Admin")

### 6.1.4 D'autre role

In [ ]:
query_Mistral7("Quelle est la capitale de la France ?", "Dev")

### 6.1.5 Hors context

In [ ]:
query_Mistral7("Quelle est la durée d'un jour sur Mars ?", "Simple")

### 6.1.6 Test zephyr

L’objectif de ce code est d’évaluer automatiquement la qualité des réponses générées par un modèle de type RAG (comme LLaMA2) en les comparant à des réponses attendues à l’aide de mesures de similarité lexicale et sémantique, afin de calculer une accuracy factuelle globale.

In [ ]:
# !pip install rapidfuzz

In [ ]:
import re, json, unicodedata
from rapidfuzz import fuzz, utils
from sentence_transformers import SentenceTransformer, util

In [ ]:
from sentence_transformers import SentenceTransformer, util
sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # 👈 nouveau nom

In [ ]:
def clean(text):
    # minuscules, accents retirés, emojis & markdown supprimés
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode()
    text = re.sub(r"`[^`]+`", " ", text)          # back‑ticks
    text = re.sub(r":[^:\s]+:", " ", text)        # :emoji:
    text = re.sub(r"[^\w\s]", " ", text)          # ponctuation
    return " ".join(text.lower().split())

def token_f1(pred, gold):
    p_tok, g_tok = pred.split(), gold.split()
    common = len(set(p_tok) & set(g_tok))
    if common == 0: return 0.0
    prec = common / len(p_tok)
    rec  = common / len(g_tok)
    return 2 * prec * rec / (prec + rec)

def score(pred, gold):
    p, g = clean(pred), clean(gold)

    # 1) containment
    if g in p: return 1.0

    # 2) token‑F1 rapide
    f1 = token_f1(p, g)
    if f1 >= 0.8: return f1          # bon seuil pour réponses courtes

    # 3) Similarité sémantique
    sim = util.cos_sim(
        sbert.encode([p])[0],      # utilise sbert, pas model !
        sbert.encode([g])[0]
    ).item()

    return sim                      # 0–1

In [ ]:
# ----------- boucle d'évaluation ----------
total, passed = 0, 0
with open("/content/eval_dataset.json", encoding="utf‑8") as f:
    data = json.load(f)

In [ ]:
for row in data:
    question   = row["question"]
    expected   = row["expected_answer"]

    generated  = query_zephyr(question, row["role"])   # ta fonction
    s          = score(generated, expected)

    total += 1
    if s >= 0.8:                   # seuil global
        passed += 1

In [ ]:
accuracy = passed / total
print(f"Accuracy factuelle : {accuracy:.2%}")

##7.1 deepseek-ai/deepseek-llm-7b-chat

In [ ]:
# from huggingface_hub import login
# login()

il est préférable de le quantifier pour optimiser les performances et réduire la consommation mémoire, surtout si tu tournes sur un GPU avec peu de VRAM.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Configuration pour charger le modèle en 4 bits
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # Utiliser float16 pour de meilleures perfs
    bnb_4bit_use_double_quant=True,  # Double quantization pour réduire encore plus la mémoire
    bnb_4bit_quant_type="nf4"  # nf4 est plus efficace que fp4
)

# Charger LLaMA-2-7B-Chat avec quantization
model_name_DeepSeek = "deepseek-ai/deepseek-llm-7b-chat"
tokenizer_DeepSeek = AutoTokenizer.from_pretrained(model_name_DeepSeek)
model_DeepSeek = AutoModelForCausalLM.from_pretrained(model_name_DeepSeek, quantization_config=quant_config, device_map="auto")

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# model_id = "deepseek-ai/deepseek-llm-7b-chat"

# tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.float16).to("cuda")

In [ ]:
# tokenizer_Llama3.pad_token_id = tokenizer_Llama3.eos_token_id

In [ ]:
# Vérifier si le modèle est bien chargé sur GPU
import torch

# Vérifie si CUDA est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Le modèle sera chargé sur : {device}")

# Envoie explicitement le modèle sur le GPU
model_DeepSeek = model_DeepSeek.to(device)

# Vérifie où se trouve une couche du modèle
print(next(model_DeepSeek.parameters()).device)

In [ ]:
# Exemple d'input (tâche explicite, ici QA)
input_text = "### Instruction:\nTraduis cette phrase en français : I love machine learning.\n### Réponse:\n"

inputs = tokenizer_DeepSeek(input_text, return_tensors="pt").to(device)
outputs = model_DeepSeek.generate(**inputs, max_new_tokens=100)
result = tokenizer_DeepSeek.decode(outputs[0], skip_special_tokens=True)
print("✅ Résultat :", result)

In [ ]:
import torch

# Charger le modèle sur le GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

def query_Model_DeepSeek(question, reponse):
    # Format the input prompt
    #formatted_prompt = f"Question: {question}\n\nRespons: {response}"
    formatted_prompt = f"""
                              Voici une question posée par un utilisateur et la réponse récupérée à partir de ChromaDB.

                              🔹 **Instructions claires :**
                              - **Si la réponse récupérée est valide**, reformule-la en français avec plus de contexte et d'explications. Ajoute des émojis dynamiquement pour illustrer les concepts (ex : 🌍 pour un lieu, 🧠 pour une explication, ✅ pour une réponse correcte). Termine la réponse par ✅.
                              - **Si la réponse est exactement "Cette question ne correspond à aucun des rôles définis.", alors affiche exactement :**
                                **"Cette question ne correspond à aucun des rôles définis. ❌"**
                              - **Si la réponse est exactement " Cette question associée à un autre rôle. ", alors affiche exactement :**
                                **"Cette question est associée à un autre rôle. 🔄"**
                              - **Ne génère aucun autre texte en dehors des trois cas définis ci-dessus.**

                              **⚠️ Attention :** Tu dois suivre ces règles strictement et ne pas ajouter d'explications supplémentaires si la réponse ne correspond pas aux données disponibles.

                              **Longueur maximale de la réponse générée : 500 caractères.**

                              Question : {question}
                              Réponse récupérée : {reponse}
                              💡 Réponse en français :
                        """


    # Tokenization
    inputs = tokenizer_DeepSeek(formatted_prompt, return_tensors="pt").to(device)

    # Génération de la réponse
    outputs = model_DeepSeek.generate(**inputs, max_new_tokens=100)

    # Décodage du résultat
    result = tokenizer_DeepSeek.decode(outputs[0], skip_special_tokens=True)

    return result

In [ ]:
llm_response = query_Model_DeepSeek("Quelle est la capitale de la France ?", "Paris")
llm_response

In [ ]:
def query_DeepSeek(query:str, role:str):
  #print('query : ', query)
  #print('role : ', role)
  response = ask_question(role, query)
  print('\nresponse ask_question : ', response)

  if response == message_question_existante_autre_role:
    return response + ' 🔄'
  elif response==message_hors_contexte:
    return response + ' ❌'
  else:
    test_answer = query_Model_DeepSeek(query, response)

    # Extraction de la réponse en français après "💡 Réponse en français :"
    start_index = test_answer.find("💡 Réponse en français :") + len("💡 Réponse en français :")
    filtered_response = test_answer[start_index:].strip()

    # Enlever "Correcte !" à la fin si nécessaire
    filtered_response = filtered_response.replace("Correct", "").strip()
    print('\nfiltered_response : ', filtered_response)

    return filtered_response

### 6.1.1 Simple Utilisateur

In [ ]:
query = "Quelle est la capitale de la France ?"
role = "Simple"

In [ ]:
Retrivel_response = ask_question(role, query)
Retrivel_response

In [ ]:
llm_response = query_Model_DeepSeek(query, Retrivel_response)
llm_response

In [ ]:
query_DeepSeek("Quelle est la capitale de la France ?", "Simple")

### 6.1.2 Dev Utilisateur

In [ ]:
query_Mistral7("Qu'est-ce qu'une API ?", "Dev")

### 6.1.3 Admin Utilisateur

In [ ]:
query_Mistral7("Comment installer Apache sur un serveur Linux ?", "Admin")

### 6.1.4 D'autre role

In [ ]:
query_Mistral7("Quelle est la capitale de la France ?", "Dev")

### 6.1.5 Hors context

In [ ]:
query_Mistral7("Quelle est la durée d'un jour sur Mars ?", "Simple")

### 6.1.6 Test DeepSeek

L’objectif de ce code est d’évaluer automatiquement la qualité des réponses générées par un modèle de type RAG (comme LLaMA2) en les comparant à des réponses attendues à l’aide de mesures de similarité lexicale et sémantique, afin de calculer une accuracy factuelle globale.

In [ ]:
# !pip install rapidfuzz

In [ ]:
import re, json, unicodedata
from rapidfuzz import fuzz, utils
from sentence_transformers import SentenceTransformer, util

In [ ]:
from sentence_transformers import SentenceTransformer, util
sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # 👈 nouveau nom

In [ ]:
def clean(text):
    # minuscules, accents retirés, emojis & markdown supprimés
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode()
    text = re.sub(r"`[^`]+`", " ", text)          # back‑ticks
    text = re.sub(r":[^:\s]+:", " ", text)        # :emoji:
    text = re.sub(r"[^\w\s]", " ", text)          # ponctuation
    return " ".join(text.lower().split())

def token_f1(pred, gold):
    p_tok, g_tok = pred.split(), gold.split()
    common = len(set(p_tok) & set(g_tok))
    if common == 0: return 0.0
    prec = common / len(p_tok)
    rec  = common / len(g_tok)
    return 2 * prec * rec / (prec + rec)

def score(pred, gold):
    p, g = clean(pred), clean(gold)

    # 1) containment
    if g in p: return 1.0

    # 2) token‑F1 rapide
    f1 = token_f1(p, g)
    if f1 >= 0.8: return f1          # bon seuil pour réponses courtes

    # 3) Similarité sémantique
    sim = util.cos_sim(
        sbert.encode([p])[0],      # utilise sbert, pas model !
        sbert.encode([g])[0]
    ).item()

    return sim                      # 0–1

In [ ]:
# ----------- boucle d'évaluation ----------
total, passed = 0, 0
with open("/content/eval_dataset.json", encoding="utf‑8") as f:
    data = json.load(f)

In [ ]:
for row in data:
    question   = row["question"]
    expected   = row["expected_answer"]

    generated  = query_DeepSeek(question, row["role"])   # ta fonction
    s          = score(generated, expected)

    total += 1
    if s >= 0.8:                   # seuil global
        passed += 1

In [ ]:
accuracy = passed / total
print(f"Accuracy factuelle : {accuracy:.2%}")

##7.1 google/flan-t5-small

In [ ]:
# from huggingface_hub import login
# login()

il est préférable de le quantifier pour optimiser les performances et réduire la consommation mémoire, surtout si tu tournes sur un GPU avec peu de VRAM.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Configuration pour charger le modèle en 4 bits
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # Utiliser float16 pour de meilleures perfs
    bnb_4bit_use_double_quant=True,  # Double quantization pour réduire encore plus la mémoire
    bnb_4bit_quant_type="nf4"  # nf4 est plus efficace que fp4
)

# Charger LLaMA-2-7B-Chat avec quantization
model_name_FlanT5 = "google/flan-t5-small"
tokenizer_FlanT5 = AutoTokenizer.from_pretrained(model_name_FlanT5)
model_FlanT5 = AutoModelForCausalLM.from_pretrained(model_name_FlanT5, quantization_config=quant_config, device_map="auto")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

In [ ]:
# tokenizer_Llama3.pad_token_id = tokenizer_Llama3.eos_token_id

In [ ]:
# Vérifier si le modèle est bien chargé sur GPU
import torch

# Vérifie si CUDA est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Le modèle sera chargé sur : {device}")

# Envoie explicitement le modèle sur le GPU
model = model.to(device)

# Vérifie où se trouve une couche du modèle
print(next(model.parameters()).device)

In [ ]:
# Exemple d'input (tâche explicite, ici QA)
input_text = "translate English to French: I love machine learning"

# Tokenization
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Génération de la réponse
outputs = model.generate(**inputs, max_length=50)

# Décodage du résultat
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("✅ Résultat :", result)

In [ ]:
import torch

# Charger le modèle sur le GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

def query_Model_FlanT5(question, reponse):
    # Format the input prompt
    #formatted_prompt = f"Question: {question}\n\nRespons: {response}"
    formatted_prompt = f"""
                              Voici une question posée par un utilisateur et la réponse récupérée à partir de ChromaDB.

                              🔹 **Instructions claires :**
                              - **Si la réponse récupérée est valide**, reformule-la en français avec plus de contexte et d'explications. Ajoute des émojis dynamiquement pour illustrer les concepts (ex : 🌍 pour un lieu, 🧠 pour une explication, ✅ pour une réponse correcte). Termine la réponse par ✅.
                              - **Si la réponse est exactement "Cette question ne correspond à aucun des rôles définis.", alors affiche exactement :**
                                **"Cette question ne correspond à aucun des rôles définis. ❌"**
                              - **Si la réponse est exactement " Cette question associée à un autre rôle. ", alors affiche exactement :**
                                **"Cette question est associée à un autre rôle. 🔄"**
                              - **Ne génère aucun autre texte en dehors des trois cas définis ci-dessus.**

                              **⚠️ Attention :** Tu dois suivre ces règles strictement et ne pas ajouter d'explications supplémentaires si la réponse ne correspond pas aux données disponibles.

                              **Longueur maximale de la réponse générée : 500 caractères.**

                              Question : {question}
                              Réponse récupérée : {reponse}
                              💡 Réponse en français :
                        """


    # Tokenization
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

    # Génération de la réponse
    outputs = model.generate(**inputs, max_length=100)

    # Décodage du résultat
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

In [ ]:
llm_response = query_Model_FlanT5("Quelle est la capitale de la France ?", "Paris")
llm_response

In [ ]:
def query_FlanT5(query:str, role:str):
  #print('query : ', query)
  #print('role : ', role)
  response = ask_question(role, query)
  print('\nresponse ask_question : ', response)

  if response == message_question_existante_autre_role:
    return response + ' 🔄'
  elif response==message_hors_contexte:
    return response + ' ❌'
  else:
    test_answer = query_Model_FlanT5(query, response)

    # Extraction de la réponse en français après "💡 Réponse en français :"
    start_index = test_answer.find("💡 Réponse en français :") + len("💡 Réponse en français :")
    filtered_response = test_answer[start_index:].strip()

    # Enlever "Correcte !" à la fin si nécessaire
    filtered_response = filtered_response.replace("Correct", "").strip()
    print('\nfiltered_response : ', filtered_response)

    return filtered_response

### 6.1.1 Simple Utilisateur

In [ ]:
query = "Quelle est la capitale de la France ?"
role = "Simple"

In [ ]:
Retrivel_response = ask_question(role, query)
Retrivel_response

In [ ]:
llm_response = query_Model_FlanT5(query, Retrivel_response)
llm_response

In [ ]:
query_FlanT5("Quelle est la capitale de la France ?", "Simple")

### 6.1.2 Dev Utilisateur

In [ ]:
query_Mistral7("Qu'est-ce qu'une API ?", "Dev")

### 6.1.3 Admin Utilisateur

In [ ]:
query_Mistral7("Comment installer Apache sur un serveur Linux ?", "Admin")

### 6.1.4 D'autre role

In [ ]:
query_Mistral7("Quelle est la capitale de la France ?", "Dev")

### 6.1.5 Hors context

In [ ]:
query_Mistral7("Quelle est la durée d'un jour sur Mars ?", "Simple")

### 6.1.6 Test  Mistral7

L’objectif de ce code est d’évaluer automatiquement la qualité des réponses générées par un modèle de type RAG (comme LLaMA2) en les comparant à des réponses attendues à l’aide de mesures de similarité lexicale et sémantique, afin de calculer une accuracy factuelle globale.

In [ ]:
# !pip install rapidfuzz

In [ ]:
import re, json, unicodedata
from rapidfuzz import fuzz, utils
from sentence_transformers import SentenceTransformer, util

In [ ]:
from sentence_transformers import SentenceTransformer, util
sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # 👈 nouveau nom

In [ ]:
def clean(text):
    # minuscules, accents retirés, emojis & markdown supprimés
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode()
    text = re.sub(r"`[^`]+`", " ", text)          # back‑ticks
    text = re.sub(r":[^:\s]+:", " ", text)        # :emoji:
    text = re.sub(r"[^\w\s]", " ", text)          # ponctuation
    return " ".join(text.lower().split())

def token_f1(pred, gold):
    p_tok, g_tok = pred.split(), gold.split()
    common = len(set(p_tok) & set(g_tok))
    if common == 0: return 0.0
    prec = common / len(p_tok)
    rec  = common / len(g_tok)
    return 2 * prec * rec / (prec + rec)

def score(pred, gold):
    p, g = clean(pred), clean(gold)

    # 1) containment
    if g in p: return 1.0

    # 2) token‑F1 rapide
    f1 = token_f1(p, g)
    if f1 >= 0.8: return f1          # bon seuil pour réponses courtes

    # 3) Similarité sémantique
    sim = util.cos_sim(
        sbert.encode([p])[0],      # utilise sbert, pas model !
        sbert.encode([g])[0]
    ).item()

    return sim                      # 0–1

In [ ]:
# ----------- boucle d'évaluation ----------
total, passed = 0, 0
with open("/content/eval_dataset.json", encoding="utf‑8") as f:
    data = json.load(f)

In [ ]:
for row in data:
    question   = row["question"]
    expected   = row["expected_answer"]

    generated  = query_Mistral7(question, row["role"])   # ta fonction
    s          = score(generated, expected)

    total += 1
    if s >= 0.8:                   # seuil global
        passed += 1

In [ ]:
accuracy = passed / total
print(f"Accuracy factuelle : {accuracy:.2%}")

##6.1 Mistral-7B-Instruct-v0.3

In [ ]:
#!pip install bitsandbytes transformers accelerate
#!pip install -U bitsandbytes

In [ ]:
# from huggingface_hub import login
# login()

il est préférable de le quantifier pour optimiser les performances et réduire la consommation mémoire, surtout si tu tournes sur un GPU avec peu de VRAM.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Configuration pour charger le modèle en 4 bits
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # Utiliser float16 pour de meilleures perfs
    bnb_4bit_use_double_quant=True,  # Double quantization pour réduire encore plus la mémoire
    bnb_4bit_quant_type="nf4"  # nf4 est plus efficace que fp4
)

# Charger LLaMA-2-7B-Chat avec quantization
model_name_Mistral7 = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer_Mistral7 = AutoTokenizer.from_pretrained(model_name_Mistral7)
model_Mistral7 = AutoModelForCausalLM.from_pretrained(model_name_Mistral7, quantization_config=quant_config, device_map="auto")

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

In [ ]:
# tokenizer_Llama3.pad_token_id = tokenizer_Llama3.eos_token_id

In [ ]:
# Vérifier si le modèle est bien chargé sur GPU
print(model_Mistral7.hf_device_map)

In [ ]:
import torch

# Charger le modèle sur le GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

def query_Model_Mistral7(question, reponse):
    # Format the input prompt
    #formatted_prompt = f"Question: {question}\n\nRespons: {response}"
    formatted_prompt = f"""
                              Voici une question posée par un utilisateur et la réponse récupérée à partir de ChromaDB.

                              🔹 **Instructions claires :**
                              - **Si la réponse récupérée est valide**, reformule-la en français avec plus de contexte et d'explications. Ajoute des émojis dynamiquement pour illustrer les concepts (ex : 🌍 pour un lieu, 🧠 pour une explication, ✅ pour une réponse correcte). Termine la réponse par ✅.
                              - **Si la réponse est exactement "Cette question ne correspond à aucun des rôles définis.", alors affiche exactement :**
                                **"Cette question ne correspond à aucun des rôles définis. ❌"**
                              - **Si la réponse est exactement " Cette question associée à un autre rôle. ", alors affiche exactement :**
                                **"Cette question est associée à un autre rôle. 🔄"**
                              - **Ne génère aucun autre texte en dehors des trois cas définis ci-dessus.**

                              **⚠️ Attention :** Tu dois suivre ces règles strictement et ne pas ajouter d'explications supplémentaires si la réponse ne correspond pas aux données disponibles.

                              **Longueur maximale de la réponse générée : 500 caractères.**

                              Question : {question}
                              Réponse récupérée : {reponse}
                              💡 Réponse en français :
                        """



    # Tokeniser l'entrée et envoyer sur le GPU
    inputs = tokenizer_Mistral7(formatted_prompt, return_tensors="pt").to(device)

    # Générer la réponse
    outputs = model_Mistral7.generate(**inputs, max_length=800)

    # Décoder et afficher la réponse
    response = tokenizer_Mistral7.decode(outputs[0], skip_special_tokens=True)

    return response

In [ ]:
def query_Mistral7(query:str, role:str):
  #print('query : ', query)
  #print('role : ', role)
  response = ask_question(role, query)
  print('\nresponse ask_question : ', response)

  if response == message_question_existante_autre_role:
    return response + ' 🔄'
  elif response==message_hors_contexte:
    return response + ' ❌'
  else:
    test_answer = query_Model_Mistral7(query, response)

    # Extraction de la réponse en français après "💡 Réponse en français :"
    start_index = test_answer.find("💡 Réponse en français :") + len("💡 Réponse en français :")
    filtered_response = test_answer[start_index:].strip()

    # Enlever "Correcte !" à la fin si nécessaire
    filtered_response = filtered_response.replace("Correct", "").strip()
    print('\nfiltered_response : ', filtered_response)

    return filtered_response

### 6.1.1 Simple Utilisateur

In [ ]:
query_Mistral7("Quelle est la capitale de la France ?", "Simple")

### 6.1.2 Dev Utilisateur

In [ ]:
query_Mistral7("Qu'est-ce qu'une API ?", "Dev")

### 6.1.3 Admin Utilisateur

In [ ]:
query_Mistral7("Comment installer Apache sur un serveur Linux ?", "Admin")

### 6.1.4 D'autre role

In [ ]:
query_Mistral7("Quelle est la capitale de la France ?", "Dev")

### 6.1.5 Hors context

In [ ]:
query_Mistral7("Quelle est la durée d'un jour sur Mars ?", "Simple")

### 6.1.6 Test  Mistral7

L’objectif de ce code est d’évaluer automatiquement la qualité des réponses générées par un modèle de type RAG (comme LLaMA2) en les comparant à des réponses attendues à l’aide de mesures de similarité lexicale et sémantique, afin de calculer une accuracy factuelle globale.

In [ ]:
# !pip install rapidfuzz

In [ ]:
import re, json, unicodedata
from rapidfuzz import fuzz, utils
from sentence_transformers import SentenceTransformer, util

In [ ]:
from sentence_transformers import SentenceTransformer, util
sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # 👈 nouveau nom

In [ ]:
def clean(text):
    # minuscules, accents retirés, emojis & markdown supprimés
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode()
    text = re.sub(r"`[^`]+`", " ", text)          # back‑ticks
    text = re.sub(r":[^:\s]+:", " ", text)        # :emoji:
    text = re.sub(r"[^\w\s]", " ", text)          # ponctuation
    return " ".join(text.lower().split())

def token_f1(pred, gold):
    p_tok, g_tok = pred.split(), gold.split()
    common = len(set(p_tok) & set(g_tok))
    if common == 0: return 0.0
    prec = common / len(p_tok)
    rec  = common / len(g_tok)
    return 2 * prec * rec / (prec + rec)

def score(pred, gold):
    p, g = clean(pred), clean(gold)

    # 1) containment
    if g in p: return 1.0

    # 2) token‑F1 rapide
    f1 = token_f1(p, g)
    if f1 >= 0.8: return f1          # bon seuil pour réponses courtes

    # 3) Similarité sémantique
    sim = util.cos_sim(
        sbert.encode([p])[0],      # utilise sbert, pas model !
        sbert.encode([g])[0]
    ).item()

    return sim                      # 0–1

In [ ]:
# ----------- boucle d'évaluation ----------
total, passed = 0, 0
with open("/content/eval_dataset.json", encoding="utf‑8") as f:
    data = json.load(f)

In [ ]:
for row in data:
    question   = row["question"]
    expected   = row["expected_answer"]

    generated  = query_Mistral7(question, row["role"])   # ta fonction
    s          = score(generated, expected)

    total += 1
    if s >= 0.8:                   # seuil global
        passed += 1

In [ ]:
accuracy = passed / total
print(f"Accuracy factuelle : {accuracy:.2%}")

##6.2 LLama-3

In [ ]:
#!pip install bitsandbytes transformers accelerate
#!pip install -U bitsandbytes

In [ ]:
# from huggingface_hub import login
# login()

il est préférable de le quantifier pour optimiser les performances et réduire la consommation mémoire, surtout si tu tournes sur un GPU avec peu de VRAM.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Configuration pour charger le modèle en 4 bits
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # Utiliser float16 pour de meilleures perfs
    bnb_4bit_use_double_quant=True,  # Double quantization pour réduire encore plus la mémoire
    bnb_4bit_quant_type="nf4"  # nf4 est plus efficace que fp4
)

# Charger LLaMA-2-7B-Chat avec quantization
model_name_Llama3 = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer_Llama3 = AutoTokenizer.from_pretrained(model_name_Llama3)
model_Llama3 = AutoModelForCausalLM.from_pretrained(model_name_Llama3, quantization_config=quant_config, device_map="auto")

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer_Llama3 = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
# model_Llama3 = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

In [ ]:
tokenizer_Llama3.pad_token_id = tokenizer_Llama3.eos_token_id

In [ ]:
# Vérifier si le modèle est bien chargé sur GPU
print(model_Llama3.hf_device_map)

In [ ]:
import torch

# Charger le modèle sur le GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

def query_Model_llama3(question, reponse):
    # Format the input prompt
    #formatted_prompt = f"Question: {question}\n\nRespons: {response}"
    formatted_prompt = f"""
                              Voici une question posée par un utilisateur et la réponse récupérée à partir de ChromaDB.

                              🔹 **Instructions claires :**
                              - **Si la réponse récupérée est valide**, reformule-la en français avec plus de contexte et d'explications. Ajoute des émojis dynamiquement pour illustrer les concepts (ex : 🌍 pour un lieu, 🧠 pour une explication, ✅ pour une réponse correcte). Termine la réponse par ✅.
                              - **Si la réponse est exactement "Cette question ne correspond à aucun des rôles définis.", alors affiche exactement :**
                                **"Cette question ne correspond à aucun des rôles définis. ❌"**
                              - **Si la réponse est exactement " Cette question associée à un autre rôle. ", alors affiche exactement :**
                                **"Cette question est associée à un autre rôle. 🔄"**
                              - **Ne génère aucun autre texte en dehors des trois cas définis ci-dessus.**

                              **⚠️ Attention :** Tu dois suivre ces règles strictement et ne pas ajouter d'explications supplémentaires si la réponse ne correspond pas aux données disponibles.

                              **Longueur maximale de la réponse générée : 500 caractères.**

                              Question : {question}
                              Réponse récupérée : {reponse}
                              💡 Réponse en français :
                        """



    # Tokeniser l'entrée et envoyer sur le GPU
    inputs = tokenizer_Llama3(formatted_prompt, return_tensors="pt").to(device)

    # Générer la réponse
    outputs = model_Llama3.generate(**inputs, max_length=800)

    # Décoder et afficher la réponse
    response = tokenizer_Llama3.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
def query_Llama3(query:str, role:str):
  #print('query : ', query)
  #print('role : ', role)
  response = ask_question(role, query)
  print('\nresponse ask_question : ', response)

  if response == message_question_existante_autre_role:
    return response + ' 🔄'
  elif response==message_hors_contexte:
    return response + ' ❌'
  else:
    test_answer = query_Model_llama3(query, response)

    # Extraction de la réponse en français après "💡 Réponse en français :"
    start_index = test_answer.find("💡 Réponse en français :") + len("💡 Réponse en français :")
    filtered_response = test_answer[start_index:].strip()

    # Enlever "Correcte !" à la fin si nécessaire
    filtered_response = filtered_response.replace("Correct", "").strip()
    print('\nfiltered_response : ', filtered_response)

    return filtered_response

### 6.1.1 Simple Utilisateur

In [ ]:
query_Llama3("Quelle est la capitale de la France ?", "Simple")

### 6.1.2 Dev Utilisateur

In [ ]:
query_Llama3("Qu'est-ce qu'une API ?", "Dev")

### 6.1.3 Admin Utilisateur

In [ ]:
query_Llama3("Comment installer Apache sur un serveur Linux ?", "Admin")

### 6.1.4 D'autre role

In [ ]:
query_Llama3("Quelle est la capitale de la France ?", "Dev")

### 6.1.5 Hors context

In [ ]:
query_Llama3("Quelle est la durée d'un jour sur Mars ?", "Simple")

### 6.1.6 Test  Llama3

L’objectif de ce code est d’évaluer automatiquement la qualité des réponses générées par un modèle de type RAG (comme LLaMA2) en les comparant à des réponses attendues à l’aide de mesures de similarité lexicale et sémantique, afin de calculer une accuracy factuelle globale.

In [ ]:
# !pip install rapidfuzz

In [ ]:
import re, json, unicodedata
from rapidfuzz import fuzz, utils
from sentence_transformers import SentenceTransformer, util

In [ ]:
from sentence_transformers import SentenceTransformer, util
sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # 👈 nouveau nom

In [ ]:
def clean(text):
    # minuscules, accents retirés, emojis & markdown supprimés
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode()
    text = re.sub(r"`[^`]+`", " ", text)          # back‑ticks
    text = re.sub(r":[^:\s]+:", " ", text)        # :emoji:
    text = re.sub(r"[^\w\s]", " ", text)          # ponctuation
    return " ".join(text.lower().split())

def token_f1(pred, gold):
    p_tok, g_tok = pred.split(), gold.split()
    common = len(set(p_tok) & set(g_tok))
    if common == 0: return 0.0
    prec = common / len(p_tok)
    rec  = common / len(g_tok)
    return 2 * prec * rec / (prec + rec)

def score(pred, gold):
    p, g = clean(pred), clean(gold)

    # 1) containment
    if g in p: return 1.0

    # 2) token‑F1 rapide
    f1 = token_f1(p, g)
    if f1 >= 0.8: return f1          # bon seuil pour réponses courtes

    # 3) Similarité sémantique
    sim = util.cos_sim(
        sbert.encode([p])[0],      # utilise sbert, pas model !
        sbert.encode([g])[0]
    ).item()

    return sim                      # 0–1

In [ ]:
# ----------- boucle d'évaluation ----------
total, passed = 0, 0
with open("/content/eval_dataset.json", encoding="utf‑8") as f:
    data = json.load(f)

In [ ]:
for row in data:
    question   = row["question"]
    expected   = row["expected_answer"]

    generated  = query_Llama3(question, row["role"])   # ta fonction
    s          = score(generated, expected)

    total += 1
    if s >= 0.8:                   # seuil global
        passed += 1


In [ ]:
accuracy = passed / total
print(f"Accuracy factuelle : {accuracy:.2%}")

##6.3 LLama-2-7b-chat via API HugginFace

In [ ]:
# !pip install bitsandbytes transformers accelerate
# !pip install -U bitsandbytes

In [47]:
from huggingface_hub import login
login()

il est préférable de le quantifier pour optimiser les performances et réduire la consommation mémoire, surtout si tu tournes sur un GPU avec peu de VRAM.

In [48]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Configuration pour charger le modèle en 4 bits
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # Utiliser float16 pour de meilleures perfs
    bnb_4bit_use_double_quant=True,  # Double quantization pour réduire encore plus la mémoire
    bnb_4bit_quant_type="nf4"  # nf4 est plus efficace que fp4
)

# Charger LLaMA-2-7B-Chat avec quantization
model_name_Llama2 = "meta-llama/Llama-2-7b-chat-hf"
tokenizer_Llama2 = AutoTokenizer.from_pretrained(model_name_Llama2)
model_Llama2 = AutoModelForCausalLM.from_pretrained(model_name_Llama2, quantization_config=quant_config, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [50]:
# Vérifier si le modèle est bien chargé sur GPU
print(model_Llama2.hf_device_map)

{'': 0}


In [51]:
import torch

# Charger le modèle sur le GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

def query_Model_llama2(question, reponse):
    # Format the input prompt
    #formatted_prompt = f"Question: {question}\n\nRespons: {response}"
    formatted_prompt = f"""
                              Voici une question posée par un utilisateur et la réponse récupérée à partir de ChromaDB.

                              🔹 **Instructions claires :**
                              - **Si la réponse récupérée est valide**, reformule-la en français avec plus de contexte et d'explications. Ajoute des émojis dynamiquement pour illustrer les concepts (ex : 🌍 pour un lieu, 🧠 pour une explication, ✅ pour une réponse correcte). Termine la réponse par ✅.
                              - **Si la réponse est exactement "Cette question ne correspond à aucun des rôles définis.", alors affiche exactement :**
                                **"Cette question ne correspond à aucun des rôles définis. ❌"**
                              - **Si la réponse est exactement " Cette question associée à un autre rôle. ", alors affiche exactement :**
                                **"Cette question est associée à un autre rôle. 🔄"**
                              - **Ne génère aucun autre texte en dehors des trois cas définis ci-dessus.**

                              **⚠️ Attention :** Tu dois suivre ces règles strictement et ne pas ajouter d'explications supplémentaires si la réponse ne correspond pas aux données disponibles.

                              **Longueur maximale de la réponse générée : 500 caractères.**

                              Question : {question}
                              Réponse récupérée : {reponse}
                              💡 Réponse en français :
                        """



    # Tokeniser l'entrée et envoyer sur le GPU
    inputs = tokenizer_Llama2(formatted_prompt, return_tensors="pt").to(device)

    # Générer la réponse
    outputs = model_Llama2.generate(**inputs, max_length=800)

    # Décoder et afficher la réponse
    response = tokenizer_Llama2.decode(outputs[0], skip_special_tokens=True)
    return response

In [53]:
def query_Llama2(query:str, role:str):
  #print('query : ', query)
  #print('role : ', role)
  response = ask_question(role, query)
  print('\nresponse ask_question : ', response)

  if response == message_question_existante_autre_role:
    return response + ' 🔄'
  elif response==message_hors_contexte:
    return response + ' ❌'
  else:
    test_answer = query_Model_llama2(query, response)

    # Extraction de la réponse en français après "💡 Réponse en français :"
    start_index = test_answer.find("💡 Réponse en français :") + len("💡 Réponse en français :")
    filtered_response = test_answer[start_index:].strip()

    # Enlever "Correcte !" à la fin si nécessaire
    filtered_response = filtered_response.replace("Correct", "").strip()
    # print('\nfiltered_response : ', filtered_response)

    return filtered_response

### 6.1.1 Simple Utilisateur

In [56]:
query_Llama2("Qui est le Groupe OMF ?", "Simple")


response ask_question :  Le Groupe OMF est une entreprise marocaine qui aide les chefs d'entreprise à relever leurs défis quotidiens.


"Le Groupe OMF est une entreprise marocaine spécialisée dans la fourniture de services de gestion d'entreprise. Ils offrent des conseils et des outils pour aider les chefs d'entreprise à relever leurs défis quotidiens et à améliorer leur gestion de l'entreprise. 🤝\n                         ✅"

### 6.1.2 Dev Utilisateur

In [57]:
query_Llama2("Qu'entend-on par optimisation au sein du Groupe OMF ?", "Dev")


response ask_question :  Amélioration de l'utilisation des ressources pour des résultats optimaux.


"L'optimisation au sein du Groupe OMF (Optimization and Maintenance Framework) est l'amélioration de l'utilisation des ressources pour obtenir les résultats optimaux. Cela implique de mettre en place des stratégies et des techniques pour maximiser l'efficacité des ressources et minimiser les coûts. L'objectif est de garantir une bonne performance, une sécurité accrue et une maintenance efficace des systèmes et des applications. ✅"

### 6.1.3 Admin Utilisateur

In [59]:
query_Llama2("Quels types de services propose le Groupe OMF ?", "Admin")


response ask_question :  Gestion, optimisation, formation, conseil, communication et technologie.


"Le Groupe OMF propose différents types de services pour aider les professionnels du marketing et de la communication à améliorer leur performance. Il offre des services de gestion, d'optimisation, de formation, de conseil, de communication et de technologie pour aider les entreprises à atteindre leurs objectifs commerciaux. 💻"

### 6.1.4 D'autre role

In [60]:
query_Llama2("Qui est le Groupe OMF ?", "Dev")


response ask_question :  Cette question associée à un autre rôle.


'Cette question associée à un autre rôle. 🔄'

### 6.1.5 Hors context

In [61]:
query_Llama2("Quelle est la durée d'un jour sur Mars ?", "Simple")


response ask_question :  Cette question ne correspond à aucun des rôles définis.


'Cette question ne correspond à aucun des rôles définis. ❌'

### 6.1.6 Test  llama2

L’objectif de ce code est d’évaluer automatiquement la qualité des réponses générées par un modèle de type RAG (comme LLaMA2) en les comparant à des réponses attendues à l’aide de mesures de similarité lexicale et sémantique, afin de calculer une accuracy factuelle globale.

In [ ]:
!pip install rapidfuzz

In [ ]:
import re, json, unicodedata
from rapidfuzz import fuzz, utils
from sentence_transformers import SentenceTransformer, util

In [ ]:
from sentence_transformers import SentenceTransformer, util
sbert = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # 👈 nouveau nom

In [ ]:
def clean(text):
    # minuscules, accents retirés, emojis & markdown supprimés
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode()
    text = re.sub(r"`[^`]+`", " ", text)          # back‑ticks
    text = re.sub(r":[^:\s]+:", " ", text)        # :emoji:
    text = re.sub(r"[^\w\s]", " ", text)          # ponctuation
    return " ".join(text.lower().split())

def token_f1(pred, gold):
    p_tok, g_tok = pred.split(), gold.split()
    common = len(set(p_tok) & set(g_tok))
    if common == 0: return 0.0
    prec = common / len(p_tok)
    rec  = common / len(g_tok)
    return 2 * prec * rec / (prec + rec)

def score(pred, gold):
    p, g = clean(pred), clean(gold)

    # 1) containment
    if g in p: return 1.0

    # 2) token‑F1 rapide
    f1 = token_f1(p, g)
    if f1 >= 0.8: return f1          # bon seuil pour réponses courtes

    # 3) Similarité sémantique
    sim = util.cos_sim(
        sbert.encode([p])[0],      # utilise sbert, pas model !
        sbert.encode([g])[0]
    ).item()

    return sim                      # 0–1

In [ ]:
# ----------- boucle d'évaluation ----------
total, passed = 0, 0
with open("/content/eval_dataset.json", encoding="utf‑8") as f:
    data = json.load(f)

In [ ]:
for row in data:
    question   = row["question"]
    expected   = row["expected_answer"]

    generated  = query_Llama2(question, row["role"])   # ta fonction
    s          = score(generated, expected)

    total += 1
    if s >= 0.8:                   # seuil global
        passed += 1


In [ ]:
accuracy = passed / total
print(f"Accuracy factuelle : {accuracy:.2%}")

# Selection Final

In [ ]:
import matplotlib.pyplot as plt

# Noms des modèles et leurs scores d'accuracy factuelle
models = ["FLAN-T5 Base/Large", "Mistral-7B", "LLaMA 3", "LLaMA 2", "GPT2", "DeepSeek"]
accuracies = [32, 61.11, 67.78, 72.48, 33.33, 68.89]  # en pourcentages

# Couleurs personnalisées : par défaut gris, sauf LLaMA 2 en vert
colors = ['gray'] * len(models)
llama2_index = models.index("LLaMA 2")
colors[llama2_index] = 'green'

# Création du graphique
plt.figure(figsize=(12, 6))
bars = plt.bar(models, accuracies, color=colors)

# Ajout des valeurs au-dessus des barres
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 1,
             f'{height:.2f}%', ha='center', va='bottom', fontsize=11)

# Détails esthétiques
plt.title("Comparaison de l'Accuracy Factuale entre différents modèles LLM", fontsize=14)
plt.ylabel("Accuracy Factuale (%)", fontsize=12)
plt.ylim(0, 80)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Annotation de justification pour LLaMA 2
plt.text(llama2_index, accuracies[llama2_index] + 4,
         "✅ Meilleur compromis : LLaMA 2", fontsize=12, color='green', ha='center')

# Affichage
plt.tight_layout()
plt.show()

# Gradio Interface

In [49]:
#!pip install gradio

In [62]:
import gradio as gr

# Création de l'interface Gradio
interface = gr.Interface(
    fn=query_Llama2,  # Appelle la fonction query(question, role)
    inputs=[
        gr.Textbox(label="Question"),  # Champ pour la question
        gr.Dropdown(["Simple", "Dev", "Admin"], label="Rôle")  # Sélection du rôle
    ],
    outputs="text",
    title="RAG Chatbot : Réponses adaptées selon le rôle",
    description="Pose une question et sélectionne un rôle (Simple, Dev ou Admin) pour obtenir une réponse personnalisée générée par LLaMa avec ChromaDB."
)

In [63]:
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d60be1b92f2d0d1f0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
interface.close()